In [1]:
%matplotlib inline
from keras.preprocessing.text import text_to_word_sequence
from __future__ import division, print_function
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return

## Load the data

In [2]:
path = "data/"
batch_size=64
max_features = 20000
maxlen = 100

In [3]:
def read_embedding_list():
    embedding_word_dict = {}
    embedding_list = []
    with open("wiki-news-300d-1M.vec", encoding="utf8") as f:
        for row in f.read().split("\n")[1:-1]:
            data = row.split(" ")
            word = data[0]
            embedding = np.array([float(num) for num in data[1:-1]])
            embedding_list.append(embedding)
            embedding_word_dict[word] = len(embedding_word_dict)

    embedding_list = np.array(embedding_list)
    return embedding_list, embedding_word_dict

In [4]:
_trainData = pd.read_csv(path+'train.csv')

In [5]:
list_sentences_train = _trainData["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]


In [6]:
_testData = pd.read_csv(path+'test.csv')

In [7]:
list_sentences_test = _testData["comment_text"].fillna("_na_").values

## Process sentences

In [8]:
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

In [9]:
def text_to_wordlist(text, remove_stopwords=True, stem_words=True):
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    
    #Replace Numbers
    text=replace_numbers.sub('n',text)
    # Clean the text, with the option to remove stopwords and to stem words.
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))

In [12]:
print(comments[0])

explanationwhi edit made usernam hardcor metallica fan revert werent vandal closur gas vote new york doll fac pleas dont remov templat talk page sinc im retir nown


In [13]:
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

## Vectorize words

In [14]:
tokenizer = Tokenizer(num_words=max_features,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)
# tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(comments + test_comments))
comments_sequence = tokenizer.texts_to_sequences(comments)

In [15]:
_labels_train = _trainData[list_classes].values
test_comments_sequence = tokenizer.texts_to_sequences(test_comments)    
_X_train = pad_sequences(comments_sequence , maxlen=maxlen)
X_train = _X_train[0:159071,:]
labels_train =_labels_train[0:159071,:]
val_train = _X_train[159071:159572,:]
val_labels_train =_labels_train[159071:159572,:]
Test_train = pad_sequences(test_comments_sequence, maxlen=maxlen)

In [16]:
print(_X_train.shape)

(159571, 100)


In [17]:
#%%bash
#wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip
#unzip wiki-news-300d-1M.vec.zip

In [18]:
embedding_list, embedding_word_dict = read_embedding_list()


MemoryError: 

In [ ]:
emb_mean,emb_std = embedding_list.mean(), embedding_list.std()
emb_mean,emb_std


In [ ]:
word_index = tokenizer.word_index
nb_words =len(word_index.items())
embed_size= embedding_list.shape[1]
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= nb_words: continue
    vec_idx = embedding_word_dict.get(word)
    if vec_idx is not None:
            embedding_vector = embedding_list[vec_idx]
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
   

In [ ]:
print(len(word_index.items()))

In [ ]:
## model

In [ ]:
#X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
#Test_train = Test_train.reshape((Test_train.shape[0], 1, Test_train.shape[1]))

In [ ]:
from attention_decoder import AttentionDecoder

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(nb_words, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = AttentionDecoder(100, embed_size)(x)
x = GlobalMaxPool1D()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(100, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
#model.load_weights("toxic.hdf")
#model.fit(X_train, labels_train, batch_size=128, epochs=1)

#y_test = model2.predict([X_te], batch_size=1024, verbose=1)

In [ ]:
#model.fit(X_train, labels_train, batch_size=128, epochs=1)

In [ ]:
#model.save_weights("model_GRU_1.h5")

In [ ]:
#preds = model.predict(Test_train)
#sample_submission = pd.read_csv(f'{path}sample_submission.csv')
#sample_submission[list_classes] = preds
#sample_submission.to_csv('submission_textgru_1.csv', index=False)

In [28]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
save_best = ModelCheckpoint('toxic.hdf', save_best_only=True, 
                           monitor='val_loss', mode='min')

history = model.fit(X_train, labels_train,batch_size=64,validation_data=(val_train, val_labels_train),
                    epochs=120, verbose=2,callbacks=[early_stopping,save_best])

Train on 159071 samples, validate on 500 samples
Epoch 1/120
 - 5648s - loss: 0.0673 - acc: 0.9781 - val_loss: 0.0316 - val_acc: 0.9877
Epoch 2/120
 - 5676s - loss: 0.0509 - acc: 0.9815 - val_loss: 0.0314 - val_acc: 0.9897
Epoch 3/120
 - 5671s - loss: 0.0451 - acc: 0.9830 - val_loss: 0.0353 - val_acc: 0.9873
Epoch 4/120
 - 5633s - loss: 0.0406 - acc: 0.9845 - val_loss: 0.0329 - val_acc: 0.9890
Epoch 5/120
 - 5603s - loss: 0.0364 - acc: 0.9859 - val_loss: 0.0385 - val_acc: 0.9870
Epoch 6/120
 - 5589s - loss: 0.0328 - acc: 0.9873 - val_loss: 0.0380 - val_acc: 0.9850
Epoch 7/120
 - 5594s - loss: 0.0296 - acc: 0.9886 - val_loss: 0.0367 - val_acc: 0.9857


In [ ]:
#model.layers[0].trainable=True

In [ ]:
#model.fit(X_train, labels_train, batch_size=64, epochs=10)

In [ ]:
#model.save_weights("model_double_GRU.h5")

In [50]:
#list_sentences_test_1 =['We are using expect for the assertions. and React Test Utils with mocha-jsdom so that we can render out some React DOM and use standard javascript commands to test our react components.']
#test_comments_1=[]
#for text in list_sentences_test_1:
#    test_comments_1.append(text_to_wordlist(text))
    
#test_comments_sequence_1 = tokenizer.texts_to_sequences(test_comments_1)   

T#est_train_1 = sequence.pad_sequences(test_comments_sequence_1, maxlen=maxlen)

In [51]:
#preds = model.predict(Test_train_1)

In [52]:
#print(np.argmax(preds, axis=1))

[0]


In [30]:
preds = model.predict(Test_train)
sample_submission = pd.read_csv(f'{path}sample_submission.csv')
sample_submission[list_classes] = preds
sample_submission.to_csv('submission_textlstm_4.csv', index=False)